[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_ban_cat.ipynb)

In [0]:
# -*- coding: utf-8 -*-
##############################################################################
#
#    OpenERP, Open Source Management Solution
#
##############################################################################
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
    def print_excel(self, cr, uid, ids, context=None):
        #set date to query_data
        #============================================================================================#        
        
        def first_day_of_month(any_day):
            any_day = parser.parse(any_day)
            return (any_day.replace(day=1)).strftime("%Y-%m-%d %H:%M:%S")

        def last_day_of_month(any_day):
            any_day = parser.parse(any_day)
            next_month = any_day.replace(day=28, hour=11, minute=59, second=59) + timedelta(days=4)
            return (next_month - timedelta(days=next_month.day)).strftime("%Y-%m-%d %H:%M:%S")
        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = first_day_of_month(date_start)
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = last_day_of_month(date_end)
        
        out_path = '/tmp/bc_ton_size_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        engine = pg.connect('postgresql://odoo:odoo@***:***/***')
        
        #Run report
        #============================================================================================# 
        def get_total(df):
            total = df.apply(np.sum)
            total['name'] = 'Total'
            df = df.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)
            return df

        def get_data(cat, date_start, date_end, engine):

            #query to get cat_id
            def get_product_id (cat):

                product_name = {'women': ('DEP NU','GIAY NU','SANDAL NU'), 'men': ('DEP NAM','GIAY NAM'), 'other':('PHU KIEN', 'PHU KIEN')}

                query='''
                    SELECT name,id as categ_id
                    FROM product_category 
                    WHERE parent_id in
                        (SELECT id 
                        FROM product_category 
                        WHERE name in %s) %s
                    ORDER BY name
                '''

                if cat == 'kid':
                    query = '''
                        SELECT name,id as categ_id
                        FROM product_category 
                        WHERE  (name like '%DEP%' AND parent_id in
                                (SELECT id 
                                FROM product_category 
                                WHERE name in ('TRE EM')))
                            OR 
                              name like 'SANDAL (KSD)'
                            OR
                               parent_id in
                                        (SELECT id 
                                        FROM product_category 
                                        WHERE name in ('GIAY TRE EM'))
                        ORDER BY name
                    '''
                elif cat == 'men':
                    query = query%(product_name['men'], "or name LIKE '%%MSD%%'")
                else:
                    query = query%(product_name[cat], "")
                return query

            #get category name and ID
            df = pd.read_sql(get_product_id(cat),engine)
            categ = tuple(df['categ_id'])

            #query to get total amount by cat and month
            get_data = '''
                SELECT foo.categ_id, foo.month, sum(foo.tong)::int as total
                FROM (
                    select t.categ_id, date_part('month', s.date_order)as month, sum(l.product_uom_qty) as tong
                    from sale_order_line l
                        join sale_order s on (l.order_id=s.id)
                        left join product_product p on (l.product_id=p.id)
                        left join product_template t on (p.product_tmpl_id=t.id)
                    where s.state not in ('draft','cancel') 
                        and (date(timezone('UTC',s.date_order::timestamp)) BETWEEN '%s' and '%s')
                        AND t.name not like '%sPQT%s'
                        AND t.categ_id in %s
                    group by 1,2
                    UNION ALL
                    select t.categ_id, date_part('month', s.date_order)as month,sum(l.qty) as tong
                    from pos_order_line l
                        join pos_order s on (l.order_id=s.id)
                        left join stock_warehouse w on (s.location_id = w.lot_stock_id)
                        left join product_product p on (l.product_id=p.id)
                            left join product_template t on (p.product_tmpl_id=t.id)
                    where s.state not in ('draft','cancel') 
                        and (date(timezone('UTC',s.date_order::timestamp)) BETWEEN '%s' and '%s')
                        AND t.name not like '%sPQT%s'
                        AND t.categ_id in %s 
                    group by 1,2) foo
                group by 1,2
                '''%(date_start,date_end,'%','%',str(categ),date_start,date_end,'%','%',str(categ))

            #get data from database
            df = pd.read_sql(get_product_id(cat),engine)
            df_3 = pd.read_sql(get_data,engine)

            #revise data to another form
            def revise_df(df):
                set_month = set(df['month'])
                final = pd.DataFrame(data={'categ_id': [i for i in set(df['categ_id'])]})
                for month in set_month:
                    month_df = df[df['month']==month].drop(['month'], axis=1)
                    month_df.columns = ['categ_id', u'Lượng bán tháng %s'%(int(month))]
                    final = pd.merge(final, month_df, how='outer', on=['categ_id'])
                return final.fillna(0).set_index('categ_id')

            pd.options.display.float_format = '{:,.0f}'.format
            df = df.set_index('categ_id')
            df_3 = revise_df(df_3)
            df = df.join(df_3)
            df = df.fillna(0)

            #create a total row for data
            df = get_total(df)

            return df

        def run_report(date_start, date_end, out_path, engine):
            
            #get data
            men = get_data('men', date_start, date_end, engine)
            women = get_data('women', date_start, date_end, engine)
            kid = get_data('kid', date_start, date_end, engine)
            other = get_data('other', date_start, date_end, engine)
            final = pd.concat([men, women, kid, other])
            final = final.reset_index(drop=True)
            
            men_sp = men[(men['name'] =='GIÀY CASUAL (MCA)') | (men['name'] =='GIÀY SNEAKER (MSN)') | (men['name'] =='GIAY THE THAO (MTT)')]
            women_sp = women[(women['name'] =='GIÀY CASUAL (WCA)') | (women['name'] =='GIÀY SNEAKER (WSN)') | (women['name'] =='GIAY THE THAO (WTT)')]
            
            #insert total row to dataframe
            men_sp = get_total(men_sp)
            women_sp = get_total(women_sp)
            
            # Insert dataframe to xlsx file
            writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
            men_sp.to_excel(writer,sheet_name = 'Final_report', startrow=4, startcol=1, index=False)
            women_sp.to_excel(writer,sheet_name = 'Final_report', startrow=9,  startcol=1, index=False)
            final.to_excel(writer,sheet_name = 'Final_report', startrow=15, startcol=1, index=False)
            
            # declare some variable
            workbook  = writer.book
            worksheet = writer.sheets['Final_report']
            big_size = workbook.add_format({'bold': True})
            big_size.set_font_size(15)
            worksheet.write('B3', u'BÁO CÁO LƯỢNG BÁN HÀNG THEO CATEGORY', big_size)

            len_men = len(men)
            len_women = len(women) + len_men
            len_kid = len(kid) + len_women
            len_other = len(other) + len_kid

            sub_total_format = workbook.add_format({'bold': True, 'font_color': 'green', 'bg_color': '#C0C0C0'})    
            header_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#B7DEE8', 'border':1})
            
            # write header and subtotal.
            for y in (len_men, len_women, len_kid, len_other):
                for x in range(final.shape[1]):
                    worksheet.write(15 + y, 1 + x, final.iloc[y-1][x], sub_total_format)

            for x in range(final.shape[1]):
                worksheet.write(8, x + 1, men_sp.iloc[3][x], sub_total_format)

            for x in range(final.shape[1]):
                worksheet.write(13, x + 1, women_sp.iloc[3][x], sub_total_format)

            for x in range(final.shape[1]):
                worksheet.write(4, x+1, final.columns.values[x], header_format)

            for x in range(final.shape[1]):
                worksheet.write(9, x+1, final.columns.values[x], header_format)

            for x in range(final.shape[1]):
                worksheet.write(15, x+1, final.columns.values[x], header_format)

            worksheet.set_column('B:B', 26)
            worksheet.set_column('C:%s'%(chr(65 +final.shape[1])), 17)

            writer.save()
        
        #rn report
        run_report(date_start, date_end, out_path, engine)
        
        #Encode final file to binary and download it.
        #============================================================================================#
        
        PREVIEW_PATH = out_path
        encoded_string = ""     
        with open(PREVIEW_PATH, "r") as image_file:
            encoded_string = base64.encodestring(image_file.read())#.b64encode
        for order in self.browse(cr, uid, ids[0]):
            order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            order.file_output = encoded_string
            data_id =  order.id
         
        return {
            'type': 'ir.actions.act_window',
            'res_model': 'report.online.performance',
            'view_mode': 'form',
            'view_type': 'form',
            'res_id': data_id,
            'views': [(False, 'form')],
            'target': 'new',
        }

# vim:expandtab:smartindent:tabstop=4:softtabstop=4:shiftwidth=4:
